## 10章：マルコフ決定過程と動的計画法

## はじめに
- これまでは識別の話(SLAM)だった
- ここからは、行動決定について扱う
- ここでは、単純な最短経路の選択(経路計画問題)とかではなく、より知的な行動決定について扱っていく
  - 経路計画問題はA*たポテンシャル法などのアルゴリズムがある
- 知的というのは、色々な状態(state)を考慮した行動(action)をとることができるということ
  - 例えば、学校をさぼるというのも知的だからできる行動

## 本章で扱う内容
- マルコフ決定過程の枠組みでロボットの行動を考える
  - 近似すれば有限マルコフ決定過程
- 有限マルコフ決定過程を解く枠組みとして動的計画法、特にここでは価値反復を扱う
- なお、これまでのSLAMの話と違って、ここでは問題の簡略化のためにロボットの真の姿勢をエージェントが知っているという前提で進める

### TOC
- 10.1:マルコフ決定過程を通した行動決定とは何かの理解
- 10.2:経路計画問題をマルコフ決定過程に当てはめながら実装
- 10.3:行動ルールを定量的に評価する方法(方策評価)
- 10.4:方策評価を拡張しより良い行動を得る方法(価値反復)
- 10.5:これまでの話をまとめたベルマン方程式の理解
- 10.6:まとめ

## 10.1 マルコフ決定過程
### マルコフ性
- 1つ前の状態と現在のアクションから、現在の状態が決まると仮定すること
  - $x_{t-2}$などの状態は影響を与えないと考えている
$$ x_t ~ p(x|x_{t-1},a_t) $$
- 現実では、マルコフ性を満たさない場合も多々あるがどうする？
  - 満たすように状態を定義する
  - 例)$x_{t-2}$まで散々動いてきたモータと$x_{t-1}$から新しくスタートするモータだと、動き(とその結果の状態)が異なる
    - こういう場合はモータの温度を$x$の変数に入れることで、マルコフ性を満たすように定義する
    - $x_{t-1}=(x,y,\theta,Mparam_1,Mparam_2)$みたいな感じ

### 評価関数
- ロボットの行動を評価する
  - なんとなく上手く動いたのは人間の主観、ロボットが自律的に動くために何が良いのかを判断する
- 評価関数(最適制御の本とかでも大体Jで定義されている)
  - $J(x_{0:T},\in R)$
    - 　x_{0:T}:状態の遷移と、a_{1:T}:行動の履歴から点数を付ける
  - それでいいのか？評価したいことを時間や危険性や消費エネルギーなどいろいろあるはず
    - 結局、行動は一つしか選べないのだから、いい感じに1つの式に評価したいことを盛り込んでスカラで扱ってしまう

### 報酬と評価
- step by stepで動くんだけど、1ステップごとに点数をつける(報酬)
- 最後のゴールにも点数を付ける(終端状態の評価)
- 全部足し合わせて評価Jとする
- $$J\left(\boldsymbol{x}_{0: T}, a_{1: T}\right)=\sum_{t=1}^T r\left(\boldsymbol{x}_{t-1}, a_t, \boldsymbol{x}_t\right)+V_{\mathrm{f}}\left(\boldsymbol{x}_T\right)$$
  - $r\left(\boldsymbol{x}_{t-1}, a_t, \boldsymbol{x}_t\right) \in R$: 状態遷移ごとに評価を与える**報酬**モデル
    - t=0からt=Tまでの状態遷移、行動履歴、報酬をまとめて**エピソード**という
  - $V_{\mathrm{f}}\left(\boldsymbol{x}_T\right)$: **終端状態**の評価

### 方策と状態価値関数
- ある初期状態$x_0$からはじめた場合、毎回報酬が変化する(状態遷移は確率的)
- では、どうやって評価するのか？
  - 行動ルールのこと(自作のプログラムのこと)を**方策**とよび$\Pi$で表す
  - 良い方策は評価Jの期待値が高いと言える
- ある初期状態$x_0$から方策$\Pi$でロボットを動かした時の$J$の期待値を**価値**といい、$V^{\Pi}(X_0)$で表す
  - もう少し砕けた言い方だと、毎回評価Jは変わるので無限回施行した時のJの平均(確率的なので期待値)を価値とした
    - 価値が高いほど良い方策のはず

### 価値の性質
- 価値を調べるといろいろ良いことがある
- $$V^{\Pi}\left(\boldsymbol{x}_0\right)=\left\langle\sum_{t=1}^T r\left(\boldsymbol{x}_{t-1}, a_t, \boldsymbol{x}_t\right)+V\left(\boldsymbol{x}_T\right)\right\rangle_{p\left(\boldsymbol{x}_{1: T}, a_{1: T} \mid \boldsymbol{x}_0, \Pi\right)}$$
  - $p\left(\boldsymbol{x}_{1: T}, a_{1: T} \mid \boldsymbol{x}_0, \Pi\right)$
    - 価値は期待値であるってことは確率分布があるわけで、ここが該当する
    - この式の意味
      - ロボットを${x}_0$におく(初期状態)、方策は$\pi$を使う、というのを条件にした時の
      - 1からT(タスクの終了)までのエピソードが${x}_{1: T}, a_{1: T}$で記述される確率
    - 多次元なので、結構厄介...
    - 毎回T変わるじゃん(いつタスクが終了するか分からない)って思うかも知れないけど、一番長いTに合わせて先に終わったエピソードは待ってもらえば、前エピソードを最長のTに統一できる(つまりTは定数で扱える)
      - 具体的には終端状態から"何もしない状態遷移かつ評価もゼロ"というアクションを繰り返して評価を先延ばしにする
      - これを使うと、T=0(初期状態=終端状態にたまたまなった場合)であっても式が破綻しないのも良い点
- ${x}_0$を分かりやすいので初期状態としたけど、別に初期状態でなくてもよい
  - なぜなら、ここではマルコフ性を前提にしているから
  - ${x}_-1$以前のことは価値に関係しないし、方策でも${x}_-1$以前を考慮する意味はない
  - よって、タスクのどのタイミングであっても、その状態に対する価値というものが考えられる
  - つまり、**状態に対して価値が一意に決まる**といえるので、これを**状態価値関数**として扱う
    - 別の表現をすると、$V^\pi:\chi\rightarrow\mathbb{R}$ということで、つまり価値$V^\pi$は状態空間$\chi$からの実数(評価は一つに決定しなければならないので、その足し合わせの価値も実数になるよね？)の写像であると考えることもできる
  
  ### 逐次式による価値表現
  - $$V^{\Pi}\left(\boldsymbol{x}_0\right)=\left\langle\sum_{t=1}^T r\left(\boldsymbol{x}_{t-1}, a_t, \boldsymbol{x}_t\right)+V\left(\boldsymbol{x}_T\right)\right\rangle_{p\left(\boldsymbol{x}_{1: T}, a_{1: T} \mid \boldsymbol{x}_0, \Pi\right)}$$
    - 価値関数を変形する(式書くの面倒なので、とりあえず変形の要点と結論だけ)
    - 初期状態とそれ以降を考える
    - 期待値を初期状態とそれ以外で分ける
    - 第一項：期待値が初期状態に関するものなので確率についても初期状態のものだけにしてそれ以外削除、第二項：確率の乗法定理で期待値の確率を初期状態とそれ以外に分ける
    - 第二項：マルコフ性より$X_1$が分かれば$X_0$と$a_1$は不要
    - 第二項：期待値の期待値のような形に持っていくと、最初の式と同じ形が再びでてくるのでその部分をt=2からTまでの価値として置き換える
      - これで式が簡単に！
    - 期待値の確率が同じなので、一つの期待値としてまとめられる
    - 乗法定理で$a_1$を分ける
    - $a_1$が条件として入っているなら方策はいらないので消す
    - $x_0$が初期状態である必要はないので、時刻を取り払う
      - 時系列ではなく遷移前後の状態を考えればよいので、前の状態$x$と後の状態$x'$で定義
  - $$V^{\mathrm{\Pi}}(\boldsymbol{x})=\left\langle r\left(\boldsymbol{x}, a, \boldsymbol{x}^{\prime}\right)+V^{\mathrm{\Pi}}\left(\boldsymbol{x}^{\prime}\right)\right\rangle_{p\left(\boldsymbol{x}^{\prime} \mid \boldsymbol{x}, a\right) P(a \mid \boldsymbol{x}, \Pi)}$$
    - 報酬と遷移後の状態の価値の期待値をとると、遷移前の価値になる
    - 逐次式にできたので、随時更新ができる！

### 決定論的方策

## 経路計画問題
- ここから、実際の実装の話にはいっていく
### 

In [1]:
# conect to drive on colab
# from google.colab import drive
# drive.mount("/content/drive")
# dir_path="./drive/MyDrive/Colab Notebooks/ProbabilisticRobotics/"
dir_path="./"

import sys
sys.path.append(dir_path)
sys.path.append('./scripts/')
from kf import *


In [2]:
class Goal:
    def __init__(self,x,y,radius=0.3):
        self.pos=np.array([x,y]).T
        self.radius=radius
    def draw(self,ax,elems):
        x,y=self.pos
        # flag
        c=ax.scatter(x+0.16,y+0.5,s=50,marker=">",label="lamdmarks",color="red")
        elems.append(c)
        # flag stick
        elems += ax.plot([x, x], [y, y + 0.6], color="black")

In [4]:
class Puddle:
    def __init__(self,lowerleft,upperright,depth):
        self.lowerleft=lowerleft
        self.upperright=upperright
        self.depth=depth
    def draw(self,ax,elems):
        w=self.upperright[0]-self.lowerleft[0]
        h=self.upperright[1]-self.lowerleft[1]
        r=patches.Rectangle(self.lowerleft,w,h,color="blue",alpha=self.depth)
        elems.append(ax.add_patch(r))

In [5]:
time_interval=0.1
world=World(30,time_interval,debug=False)

# add map and landmarks
m=Map()
for ln in [(-4,2),(2,-3),(3,3)]:m.append_landmark(Landmark(*ln))
world.append(m)

# add Gpal
world.append(Goal(-3,-3))

# add puddle
world.append(Puddle((-2,0),(0,2),0.1))
world.append(Puddle((-0.5,-2),(2.5,1),0.1))

# add robot
init_pose=np.array([0,0,0]).T
kf=KalmanFilter(m,init_pose)
a=EstimationAgent(time_interval,0.2,10/180*np.pi,kf)
r=Robot(init_pose,sensor=Camera(m,distance_bias_rate_stddev=0,direction_bias_stddev=0),
        agent=a,color="red",bias_rate_stds=(0,0))
world.append(r)

world.draw()
